Processing Data

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

connection = sqlite3.connect("focus.db")
df = pd.read_sql("SELECT * FROM focus_logs",connection)
#The first thing is to add a tracking ended entry. So...
df

,id,date,time,focused,program,session_end
0,1,09/09/2024,22:21:09,focus.py - Time-Manager - Visual Studio Code,Code.exe,0
1,2,09/09/2024,22:21:12,Efficient Time Calculation - Google Chrome,chrome.exe,0
2,3,09/09/2024,22:21:15,focus.py - Time-Manager - Visual Studio Code,Code.exe,0
3,4,09/09/2024,22:21:17,Task Switching,explorer.exe,0
4,5,09/09/2024,22:21:18,Chris Chan's Horrifying History (Full Document...,chrome.exe,0
5,6,09/09/2024,22:21:20,focus.py - Time-Manager - Visual Studio Code,Code.exe,0
6,7,09/09/2024,22:21:20,Session End,None,1


In [2]:
programInsights = pd.read_sql("SELECT * FROM program_insights",connection)
#The first thing is to add a tracking ended entry. So...
programInsights


,id,program,total_time,average_time,context_switch,last_updated
0,1,Code.exe,0 days 00:00:05,0 days 00:00:01.666666666,3,2024-09-10 02:21:20
1,2,chrome.exe,0 days 00:00:05,0 days 00:00:02.500000,2,2024-09-10 02:21:20
2,3,explorer.exe,0 days 00:00:01,0 days 00:00:01,1,2024-09-10 02:21:20
3,4,None,0 days 00:00:05,0 days 00:00:05,1,2024-09-10 02:21:20


In [3]:
generalInsights = pd.read_sql("SELECT * FROM general_insights",connection)
#The first thing is to add a tracking ended entry. So...
generalInsights

,id,total_context_switches,last_updated


In [4]:
df = pd.read_sql("SELECT * FROM focus_logs",connection)
#The first thing is to add a tracking ended entry. So...
# Now by entry, I want to get a total time/add up datetimes. Join "Date" and "Time" then just sum up?
df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])
df = df.sort_values(by=['datetime'])

# Current datetime - next datetime
df['timespent'] = df['datetime'].shift(-1) - df['datetime']
# For the last row, set a default time (e.g., 5 seconds), as there's no next row
df.iloc[-1, df.columns.get_loc('timespent')] = pd.Timedelta(seconds=5)

total_time_per_program = df.groupby('program')['timespent'].sum()
df

,id,date,time,focused,program,session_end,datetime,timespent
0,1,09/09/2024,22:15:49,● focus.py - Time-Manager - Visual Studio Code,Code.exe,0,2024-09-09 22:15:49,0 days 00:00:01
1,2,09/09/2024,22:15:50,Session End,None,1,2024-09-09 22:15:50,0 days 00:00:04
2,3,09/09/2024,22:15:54,● focus.py - Time-Manager - Visual Studio Code,Code.exe,0,2024-09-09 22:15:54,0 days 00:00:05
3,4,09/09/2024,22:15:59,Session End,None,1,2024-09-09 22:15:59,0 days 00:00:06
4,5,09/09/2024,22:16:05,focus.py - Time-Manager - Visual Studio Code,Code.exe,0,2024-09-09 22:16:05,0 days 00:00:00
5,6,09/09/2024,22:16:05,Session End,None,1,2024-09-09 22:16:05,0 days 00:00:05


In [10]:
import pandas as pd
import sqlite3

# Load data from the database
connection = sqlite3.connect("focus.db")
df = pd.read_sql("SELECT * FROM focus_logs", connection)

# Convert 'date' and 'time' columns into a single datetime column
df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])
df = df.sort_values(by=['datetime'])

# Initialize dictionaries to hold total time and counts per program
total_time_per_program = {}
program_switch_counts = {}

# Iterate through the dataframe
previous_program = None
program_start_time = None

for i, row in df.iterrows():
    current_program = row['program']
    
    if previous_program is None:  # First program encountered
        previous_program = current_program
        program_start_time = row['datetime']
        continue

    # If the program changes or session ends, calculate time spent on the previous program
    if current_program != previous_program or row.get('session_end', False):
        # Calculate time spent on the previous program
        time_spent = row['datetime'] - program_start_time

        # Update total time and switch count for the previous program
        if previous_program not in total_time_per_program:
            total_time_per_program[previous_program] = time_spent
            program_switch_counts[previous_program] = 1
        else:
            total_time_per_program[previous_program] += time_spent
            program_switch_counts[previous_program] += 1

        # Reset for the new program
        previous_program = current_program
        program_start_time = row['datetime']

# Handle the last program
# Assuming a small default time if the last entry is not followed by a session end
if previous_program is not None:
    # Set a default value for the last entry
    time_spent = pd.Timedelta(seconds=5)
    if previous_program not in total_time_per_program:
        total_time_per_program[previous_program] = time_spent
        program_switch_counts[previous_program] = 1
    else:
        total_time_per_program[previous_program] += time_spent
        program_switch_counts[previous_program] += 1

# Calculate average time per program
average_time_per_program = {
    program: total_time / count 
    for program, (total_time, count) in zip(total_time_per_program.keys(), 
                                             zip(total_time_per_program.values(), 
                                                 program_switch_counts.values()))
}

# Output total and average times
print("Total Time Per Program:")
for program, total_time in total_time_per_program.items():
    print(f"{program}: {total_time}")

print("\nAverage Time Per Program:")
for program, average_time in average_time_per_program.items():
    print(f"{program}: {average_time}")

Total Time Per Program:
Code.exe: 0 days 00:00:03
chrome.exe: 0 days 00:00:03
None: 0 days 00:00:05

Average Time Per Program:
Code.exe: 0 days 00:00:01.500000
chrome.exe: 0 days 00:00:03
None: 0 days 00:00:05


In [8]:
# # #First DF

# # # At the beginning of all of this, in the raw csv, add a entry signifying the Tracking Ended. This is so we don't have
# # # crazy gaps between ending and starting the next session. Add an ignorance for it.
# # from datetime import datetime
# # # Now by entry, I want to get a total time/add up datetimes. Join "Date" and "Time" then just sum up?
# df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])
# #Makes a column w full datetime object

# # Sort by 'DateTime' to ensure correct time difference calculation
# df = df.sort_values(by=['datetime'])

# # Current datetime - next datetime
# df['timespent'] = df['datetime'].shift(-1) - df['datetime']
# # For the last row, set a default time (e.g., 5 seconds), as there's no next row
# df.iloc[-1, df.columns.get_loc('timespent')] = pd.Timedelta(seconds=5)

# # Group by 'Program' and sum the total time spent for each program
# total_time_per_program = df.groupby('program')['timespent'].sum()
# print(total_time_per_program.sort_values(ascending=False))


program
chrome.exe     0 days 00:00:13
Code.exe       0 days 00:00:08
explorer.exe   0 days 00:00:02
Name: timespent, dtype: timedelta64[ns]


In [10]:
# #Task: Storing insights into the database
# insights_data = {
#     'total_time': total_time_per_program
# }
# cursor = connection.cursor()

# # Insert or update insights for each program
# for program, total_time in total_time_per_program.items():

#     # Insert or update the row for the program
#     cursor.execute('''
#         INSERT INTO program_insights (program, total_time)
#         VALUES (?, ?)
#         ON CONFLICT(program) DO UPDATE SET 
#         total_time = excluded.total_time,
#         last_updated = CURRENT_TIMESTAMP
#     ''', (program, str(total_time)))
